In [1]:
# Installing Tensorflow
# pip install tensorflow

# Installing Keras
# pip install --upgrade keras


In [2]:
# Part 1 - Data Preprocessing

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd


In [3]:
# Importing the dataset
dataset = pd.read_csv("Churn_Modelling.csv")

In [4]:
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
df=pd.get_dummies(columns=["Geography","Gender"],data=dataset)

In [6]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain,Gender_Female,Gender_Male
0,1,15634602,Hargrave,619,42,2,0.00,1,1,1,101348.88,1,1,0,0,1,0
1,2,15647311,Hill,608,41,1,83807.86,1,0,1,112542.58,0,0,0,1,1,0
2,3,15619304,Onio,502,42,8,159660.80,3,1,0,113931.57,1,1,0,0,1,0
3,4,15701354,Boni,699,39,1,0.00,2,0,0,93826.63,0,1,0,0,1,0
4,5,15737888,Mitchell,850,43,2,125510.82,1,1,1,79084.10,0,0,0,1,1,0


In [7]:
df=df.drop(["RowNumber","CustomerId","Surname"],axis=1)

In [8]:
X = df.drop("Exited",axis=1)
y = df[["Exited"]]

In [9]:
X.shape

(10000, 13)

In [10]:
# Splitting the dataset into the Training set and Test set
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [11]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
# Part 2 - Now let's make the ANN!

# Importing the Keras libraries and packages
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense


In [13]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=13))

# Adding the second hidden layer
classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))

# Adding the output layer
classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))

# Compiling the ANN
classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Fitting the ANN to the Training set
classifier.fit(X_train, y_train, batch_size=10, epochs=10, validation_split=0.1)

Epoch 1/10
720/720 [==============================] - 8s 7ms/step - loss: 0.5024 - accuracy: 0.7956 - val_loss: 0.4132 - val_accuracy: 0.7950
Epoch 2/10
720/720 [==============================] - 4s 5ms/step - loss: 0.4189 - accuracy: 0.8133 - val_loss: 0.3922 - val_accuracy: 0.8300
Epoch 3/10
720/720 [==============================] - 4s 5ms/step - loss: 0.4066 - accuracy: 0.8219 - val_loss: 0.3776 - val_accuracy: 0.8375
Epoch 4/10
720/720 [==============================] - 4s 5ms/step - loss: 0.3968 - accuracy: 0.8247 - val_loss: 0.3683 - val_accuracy: 0.8388
Epoch 5/10
720/720 [==============================] - 4s 5ms/step - loss: 0.3899 - accuracy: 0.8264 - val_loss: 0.3619 - val_accuracy: 0.8375
Epoch 6/10
720/720 [==============================] - 4s 5ms/step - loss: 0.3843 - accuracy: 0.8267 - val_loss: 0.3580 - val_accuracy: 0.8487
Epoch 7/10
720/720 [==============================] - 4s 6ms/step - loss: 0.3797 - accuracy: 0.8296 - val_loss: 0.3563 - val_accuracy: 0.8388
Epoch 

In [14]:
# Part 3 - Making predictions and evaluating the model

# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred = (y_pred > 0.5)

# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)
print(cm)



63/63 [==============================] - 1s 4ms/step
[[1530   65]
 [ 226  179]]


### - Put me one step down on the podium

In [ ]:
#from tensorflow.keras.wrappers.scikit_learn import KerasClassifier


In [15]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.layers import Dense, Dropout

In [16]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu', input_dim=13))
    classifier.add(Dropout(0.1))
    classifier.add(Dense(units=6, kernel_initializer='uniform', activation='relu'))
    classifier.add(Dropout(0.1))
    classifier.add(Dense(units=1, kernel_initializer='uniform', activation='sigmoid'))
    classifier.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    return classifier


classifier = KerasClassifier(build_fn=build_classifier)
parameters = {'batch_size': [128, 256, 512],
              'epochs': [30, 50],
              'optimizer': ['adam']}
grid_search = GridSearchCV(estimator=classifier,
                           param_grid=parameters,
                           scoring='accuracy',
                           cv=10)
grid_search = grid_search.fit(X_train, y_train)
best_parameters = grid_search.best_params_
best_accuracy = grid_search.best_score_
print("Best parameters:", best_parameters)
print("Best accuracy:", best_accuracy)

C:\Users\Premalatha\AppData\Local\Temp\ipykernel_20312\4017053125.py:12: DeprecationWarning: KerasClassifier is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  classifier = KerasClassifier(build_fn=build_classifier)


Epoch 1/30
57/57 [==============================] - 4s 6ms/step - loss: 0.6835 - accuracy: 0.7871
Epoch 2/30
57/57 [==============================] - 0s 6ms/step - loss: 0.6474 - accuracy: 0.7971
Epoch 3/30
57/57 [==============================] - 0s 6ms/step - loss: 0.5687 - accuracy: 0.7971
Epoch 4/30
57/57 [==============================] - 0s 6ms/step - loss: 0.4935 - accuracy: 0.7971
Epoch 5/30
57/57 [==============================] - 0s 6ms/step - loss: 0.4636 - accuracy: 0.7971
Epoch 6/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4550 - accuracy: 0.7971
Epoch 7/30
57/57 [==============================] - 0s 6ms/step - loss: 0.4514 - accuracy: 0.7971
Epoch 8/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4466 - accuracy: 0.7971
Epoch 9/30
57/57 [==============================] - 0s 6ms/step - loss: 0.4453 - accuracy: 0.7971
Epoch 10/30
57/57 [==============================] - 0s 6ms/step - loss: 0.4434 - accuracy: 0.7971
Epoch 11/30
57/57 [

Epoch 23/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4290 - accuracy: 0.7956
Epoch 24/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4317 - accuracy: 0.7956
Epoch 25/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4281 - accuracy: 0.7956
Epoch 26/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4293 - accuracy: 0.7956
Epoch 27/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4289 - accuracy: 0.7956
Epoch 28/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4257 - accuracy: 0.7956
Epoch 29/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4273 - accuracy: 0.7956
Epoch 30/30
25/25 [==============================] - 0s 4ms/step
Epoch 1/30
57/57 [==============================] - 4s 5ms/step - loss: 0.6796 - accuracy: 0.7972
Epoch 2/30
57/57 [==============================] - 0s 5ms/step - loss: 0.6126 - accuracy: 0.7975
Epoch 3/30
57/57 [============================

57/57 [==============================] - 0s 5ms/step - loss: 0.4384 - accuracy: 0.7944
Epoch 15/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4441 - accuracy: 0.7944
Epoch 16/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4411 - accuracy: 0.7944
Epoch 17/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4403 - accuracy: 0.7944
Epoch 18/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4411 - accuracy: 0.7944
Epoch 19/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4427 - accuracy: 0.7944
Epoch 20/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4426 - accuracy: 0.7944
Epoch 21/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4383 - accuracy: 0.7944
Epoch 22/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4396 - accuracy: 0.7944
Epoch 23/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4426 - accuracy: 0.7944
Epoch 24/30
57/57 [===

57/57 [==============================] - 0s 5ms/step - loss: 0.4507 - accuracy: 0.7956
Epoch 6/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4401 - accuracy: 0.8022
Epoch 7/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4363 - accuracy: 0.8115
Epoch 8/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4348 - accuracy: 0.8126
Epoch 9/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4309 - accuracy: 0.8154
Epoch 10/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4269 - accuracy: 0.8190
Epoch 11/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4258 - accuracy: 0.8196
Epoch 12/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4261 - accuracy: 0.8213
Epoch 13/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4224 - accuracy: 0.8256
Epoch 14/30
57/57 [==============================] - 0s 5ms/step - loss: 0.4207 - accuracy: 0.8210
Epoch 15/30
57/57 [=======

...

Epoch 22/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4508 - accuracy: 0.7937
Epoch 23/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4476 - accuracy: 0.7937
Epoch 24/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4493 - accuracy: 0.7937
Epoch 25/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4475 - accuracy: 0.7937
Epoch 26/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4480 - accuracy: 0.7937
Epoch 27/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4479 - accuracy: 0.7937
Epoch 28/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4480 - accuracy: 0.7937
Epoch 29/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4453 - accuracy: 0.7937
Epoch 30/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4448 - accuracy: 0.7937
Epoch 31/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4455 - accuracy: 0.7937
Epoch 32/5

15/15 [==============================] - 0s 6ms/step - loss: 0.6810 - accuracy: 0.7969
Epoch 4/50
15/15 [==============================] - 0s 6ms/step - loss: 0.6750 - accuracy: 0.7969
Epoch 5/50
15/15 [==============================] - 0s 6ms/step - loss: 0.6675 - accuracy: 0.7969
Epoch 6/50
15/15 [==============================] - 0s 6ms/step - loss: 0.6569 - accuracy: 0.7969
Epoch 7/50
15/15 [==============================] - 0s 6ms/step - loss: 0.6403 - accuracy: 0.7969
Epoch 8/50
15/15 [==============================] - 0s 6ms/step - loss: 0.6170 - accuracy: 0.7969
Epoch 9/50
15/15 [==============================] - 0s 6ms/step - loss: 0.5880 - accuracy: 0.7969
Epoch 10/50
15/15 [==============================] - 0s 6ms/step - loss: 0.5570 - accuracy: 0.7969
Epoch 11/50
15/15 [==============================] - 0s 6ms/step - loss: 0.5277 - accuracy: 0.7969
Epoch 12/50
15/15 [==============================] - 0s 6ms/step - loss: 0.5040 - accuracy: 0.7969
Epoch 13/50
15/15 [=========

15/15 [==============================] - 0s 7ms/step - loss: 0.4456 - accuracy: 0.7962
Epoch 36/50
15/15 [==============================] - 0s 7ms/step - loss: 0.4479 - accuracy: 0.7962
Epoch 37/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4433 - accuracy: 0.7962
Epoch 38/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4422 - accuracy: 0.7962
Epoch 39/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4422 - accuracy: 0.7962
Epoch 40/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4424 - accuracy: 0.7962
Epoch 41/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4440 - accuracy: 0.7962
Epoch 42/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4428 - accuracy: 0.7962
Epoch 43/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4430 - accuracy: 0.7962
Epoch 44/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4449 - accuracy: 0.7962
Epoch 45/50
15/15 [===

15/15 [==============================] - 0s 9ms/step - loss: 0.4535 - accuracy: 0.7961
Epoch 17/50
15/15 [==============================] - 0s 10ms/step - loss: 0.4531 - accuracy: 0.7961
Epoch 18/50
15/15 [==============================] - 0s 10ms/step - loss: 0.4505 - accuracy: 0.7961
Epoch 19/50
15/15 [==============================] - 0s 8ms/step - loss: 0.4517 - accuracy: 0.7961
Epoch 20/50
15/15 [==============================] - 0s 9ms/step - loss: 0.4486 - accuracy: 0.7961
Epoch 21/50
15/15 [==============================] - 0s 8ms/step - loss: 0.4478 - accuracy: 0.7961
Epoch 22/50
15/15 [==============================] - 0s 7ms/step - loss: 0.4464 - accuracy: 0.7961
Epoch 23/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4469 - accuracy: 0.7961
Epoch 24/50
15/15 [==============================] - 0s 6ms/step - loss: 0.4452 - accuracy: 0.7961
Epoch 25/50
15/15 [==============================] - 0s 7ms/step - loss: 0.4425 - accuracy: 0.7961
Epoch 26/50
15/15 [=

63/63 [==============================] - 0s 6ms/step - loss: 0.3752 - accuracy: 0.8464
Epoch 49/50
63/63 [==============================] - 0s 6ms/step - loss: 0.3722 - accuracy: 0.8476
Epoch 50/50
63/63 [==============================] - 0s 6ms/step - loss: 0.3737 - accuracy: 0.8479
Best parameters: {'batch_size': 128, 'epochs': 50, 'optimizer': 'adam'}
Best accuracy: 0.817375
